In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


In [ ]:
print(spark.version)
#3.3.2

3.3.2


In [4]:
file=r'C:\Users\Guanyi\DE_Zoom_Camp\Week5\yellow_tripdata_2024-10.parquet'
df=spark.read.parquet(file)
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [ ]:
df=df.repartition(4)
df.write.parquet('homework5/')
#avg size 25mb

In [17]:
file=r'C:\Users\Guanyi\DE_Zoom_Camp\Week5\yellow_tripdata_2024-10.parquet'
df_yellow=spark.read.parquet(file)
df_yellow.createOrReplaceTempView('yellow_data')

In [18]:
spark.sql("""
SELECT
    count(*)
FROM
    yellow_data
WHERE DATE(tpep_pickup_datetime) = '2024-10-15'


""").show()

+--------+
|count(1)|
+--------+
|  128632|
+--------+



In [20]:
spark.sql("""
SELECT 
    (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS duration_hrs
FROM yellow_data
ORDER BY duration_hrs DESC
""").show()



+------------------+
|      duration_hrs|
+------------------+
|162.61777777777777|
|           143.325|
|137.76055555555556|
|114.83472222222223|
| 89.89833333333333|
| 89.44611111111111|
| 70.29916666666666|
| 67.57333333333334|
| 66.06666666666666|
|           46.4225|
| 42.30888888888889|
| 38.47416666666667|
| 33.95111111111111|
| 26.29861111111111|
| 25.29138888888889|
|25.238333333333333|
|             24.47|
|23.996666666666666|
|23.995277777777776|
|23.994722222222222|
+------------------+
only showing top 20 rows



In [24]:
zone_file=r'C:\Users\Guanyi\DE_Zoom_Camp\Week5\taxi_zone_lookup.csv'
df_zone = spark.read \
    .option("header", "true") \
    .csv(zone_file)

In [25]:
df_zone.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



localhost:4040

In [31]:
df_result=df_yellow.join(df_zone,df_zone.LocationID==df_yellow.PULocationID)
df_result.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone

In [32]:
df_result.createOrReplaceTempView('trips_with_zones')

In [35]:
spark.sql("""
SELECT 
    Zone,
    count(*) as total_trips
FROM
    trips_with_zones
group by Zone
order by total_trips 


""").show()

+--------------------+-----------+
|                Zone|total_trips|
+--------------------+-----------+
|Governor's Island...|          1|
|       Arden Heights|          2|
|       Rikers Island|          2|
|         Jamaica Bay|          3|
| Green-Wood Cemetery|          3|
|Charleston/Totten...|          4|
|       Port Richmond|          4|
|   Rossville/Woodrow|          4|
|Eltingville/Annad...|          4|
|       West Brighton|          4|
|        Crotona Park|          6|
|         Great Kills|          6|
|Heartland Village...|          7|
|     Mariners Harbor|          7|
|Saint George/New ...|          9|
|             Oakwood|          9|
|New Dorp/Midland ...|         10|
|       Broad Channel|         10|
|         Westerleigh|         12|
|     Pelham Bay Park|         12|
+--------------------+-----------+
only showing top 20 rows

